In [9]:
import numpy as np
from scipy.integrate import odeint
import math
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit


π=np.pi
G=4.4908e-15            #in units of M☉^-1 pc^3 yr^-2
ρ_eq=3.1811e3           #in units of M☉ pc^-3 with ρ_eq=2.15368e-16 kg m^-3
t_eq=5.0506e4           #in units of yr for t_eq = 1.59246e12 seconds


M_1PBH_list=[1,30,100,1000]             #in units of M☉  
M_2PBH_list=[10,50,100,10]              #in units of M☉


x=0.063     # in units of pc              # let say


λ_list=np.zeros(len(M_1PBH_list))

def λ_0(M_1PBH,M_2PBH):
     return ((8*π*ρ_eq*(x**3))/(3*(M_1PBH+M_2PBH)))

for k, (M_1PBH,M_2PBH) in enumerate (zip(M_1PBH_list,M_2PBH_list)):
     λ_list[k]=λ_0(M_1PBH,M_2PBH) 

        
        
def h(x):
    return math.sqrt((x**-3)+(x**-4))   
def g(x):
    return  -((3*x+4)/(2*h(x)*(x**5))) #g(x)=h'(x).
    
    
def s_dec(λ_0):
    def f(u,s,λ_0):
            return [u[1], -((s*g(s)+h(s))*((s*u[1]-u[0]))/(h(s)*s**2))-((u[0])/((((h(s)*s)**2))*λ_0
                     *((u[0]**2+b**2)**(3/2))))]    #eq. of the dynamics of the PBH binary
    separation0= [1e-12,1]
    b= (3*λ_0)/100000
    ss= np.linspace(1e-12,((6*λ_0)/3),100000) 
    us= odeint(f,separation0,ss,args=( λ_0,))
    separations= us[:,0]
    As=ss/λ_0
    Bs=separations/λ_0
    Bs_list = Bs.tolist()
    As_list = As.tolist()
    for i in  range(1,len(Bs_list)):
        Δ=(Bs[i] - Bs[i-1])
        s=(As[i]*λ_0)              #s/λ at which the binary decouples.
        A=Bs[i] 
        if Δ<0:
            break
    return s


def semi(λ_0):
    def f(u,s,λ_0):
        return [u[1], -((s*g(s)+h(s))*((s*u[1]-u[0]))/(h(s)*s**2))-((u[0])/((((h(s)*s)**2))*λ_0
                     *((u[0]**2+b**2)**(3/2))))]    
    separation0= [1e-12,1]
    b=(3*λ_0)/100000
    ss= np.linspace(1e-12,(2*λ_0),1000000) 
    us= odeint(f,separation0,ss,args=( λ_0,))
    separations= us[:,0]
    As=ss/λ_0
    Bs=separations/λ_0
    Bs_list = Bs.tolist()
    As_list = As.tolist()
    for i in  range(1,len(Bs_list)):
        Δ=(Bs[i] - Bs[i-1])
        A=Bs[i]
        a=((A/2)*λ_0)                   #as Bs=separations/λ_0.
        if Δ<0:
            break
    
    return a


s_dec_list=np.zeros(len(λ_list))
a_list=np.zeros(len(λ_list))


for j, λ_0 in enumerate (λ_list):
    s_dec_list[j]=s_dec(λ_0)
    a_list[j]=semi(λ_0)
    
print("a_i=",a_list,"pc")


e_list=[0.9999,0.999,0.99,0.9]        
        
def r_min(e_i,a_i):
    return (a_i*(1-e_i)) 

r_min_list=np.zeros(len(a_list))
for k, (e_i,a_i) in enumerate (zip(e_list,a_list)):
     r_min_list[k]=r_min(e_i,a_i)
        
print("r_min=",r_min_list,"pc")        
        

def t_dec(s_dec):
    return (((3/(8*π*G*ρ_eq))**(1/4))*(((2/3)*(s_dec-2)*((s_dec+1)**(1/2))+(4/3))**(1/2)))

def r_ta(M_PBH,s_dec): 
    return ((4*G*M_PBH*t_dec(s_dec))**(1/3))

r_ta_M_1PBH_list=np.zeros(len(λ_list))
r_ta_M_2PBH_list=np.zeros(len(λ_list))

for j, (M_PBH,s_dec) in enumerate (zip(M_1PBH_list,s_dec_list)):
    r_ta_M_1PBH_list[j]=r_ta(M_PBH,s_dec)
    
for j, (M_PBH,s_dec) in enumerate (zip(M_2PBH_list,s_dec_list)):
    r_ta_M_2PBH_list[j]=r_ta(M_PBH,s_dec)    
    
print("r_ta_M_1PBH=",r_ta_M_1PBH_list,"pc")
print("r_ta_M_2PBH=",r_ta_M_2PBH_list,"pc") 



a_i= [0.06385811 0.0082129  0.00326448 0.00064426] pc
r_min= [6.38581129e-06 8.21289916e-06 3.26447535e-05 6.44255064e-05] pc
r_ta_M_1PBH= [9.47890438e-05 1.49059478e-04 1.63752196e-04 2.05428573e-04] pc
r_ta_M_2PBH= [2.04216804e-04 1.76729553e-04 1.63752196e-04 4.42582444e-05] pc
